In [1]:
# Llibreries que farem servir
import os
import requests
import csv
import re
import datetime, calendar
from datetime import timedelta
from bs4 import BeautifulSoup
import pandas as pd



In [2]:
# Funció per fer el dataframe d'un mes
def __audiencia_del_mes(anyo, mes):
    """Registre de la audiencia del mes de l'any indicat"""
    
    # dataframe on acumularem un mes
    data = pd.DataFrame(columns=('Audiencia', 'Cadena', 'Dia', 'Mes','Any'))
    
    # Interval temporal que farem servir
    year = anyo
    month = mes

    # Càlcul del calendari
    num_days = calendar.monthrange(year, month)[1]
    days = [datetime.date(year, month, day) for day in range(1, num_days+1)]
    
    # Extracció de les dades 
    for day in days: 
        # Reconstuim la data de la url per dia
        fecha = day.strftime('%Y-%m-%d')
        url = 'https://www.formulatv.com/audiencias/' + fecha
        
        # Fem el request de l'script de la pàrgina de la web 
        req = requests.get(url, timeout=10)
        soup = BeautifulSoup(req.text, "lxml")

        # Busquem els elements "ul" 
        ul = soup.h1.find_next_sibling("ul")
        cadenas1 = (" ".join([img['alt'] for img in ul.find_all('img', alt=True)]))
        cadenas1 = re.sub(re.compile('logo'),"",cadenas1)

        # Ho passem a llista
        cadenas1 = cadenas1.split(" ")
        
        # Treiem el percentatges, és a dir, el text de cada "li"
        # Escollim posar un espai com a separador
        audiencias1 = ul.get_text(' ', strip=True)
        
        # Ho passem a llista
        audiencias1 = audiencias1.split(" ")
        
        # Aquestes llistes seran les columnes del nostre dataframse
        audiencias = pd.DataFrame(audiencias1, columns=['Audiencia'])
        cadenas = pd.DataFrame(cadenas1, columns=['Cadena'])

        # Afegim la informació temporal separada
        # D'aquesta forma ens serà més fàcil posteriorment tractar-la
        df = pd.concat([cadenas, audiencias], axis=1, sort=False) 
        df["Dia"] = day.strftime('%d')
        df["Mes"] = day.strftime('%m')
        df["Any"] = day.strftime('%Y')
        
        data = pd.concat([data, df], sort=False) 
    
    return data

In [3]:
# Iniciatlitzem el data frame
data_1Q_2019 = pd.DataFrame(columns=('Audiencia', 'Cadena', 'Dia', 'Mes','Any'))

# Escollim l'any
year = 2019

# 1er Quatrimestre
for new_month in range(1, 5):
    df = __audiencia_del_mes(year, new_month)
    data_1Q_2019  = pd.concat([data_1Q_2019, df], sort=False) 


In [11]:
# Reordenem les columnes
data_1Q_2019 = data_1Q_2019[['Audiencia', 'Cadena', 'Dia', 'Mes','Any']]

# Reiniciem el índex
data_1Q_2019 = data_1Q_2019.reset_index()

# Renombrem la columna on tindrem el ranking diari
data_1Q_2019 = data_1Q_2019.rename(columns={'index': 'Ranking diari'})

# Volquem el dataframe en un fitxer CSV
# Construim l'adreça on s'emmagatzemarà la informació
cwd = os.getcwd()
local_path = cwd + '\export_1Q_year_'+str(year) + '.csv'
data_1Q_2019.to_csv (local_path, index = False, header=True)
